

## 1)  Loading and cleaning raw dataset 
In [SLNI_dataset](https://raw.githubusercontent.com/Jivnesh/ISCLS-19/master/data/sample_snli.jsonl), each line containes one data-sample.





In [69]:
import json
import numpy as np
from tqdm import tqdm
!wget https://raw.githubusercontent.com/Jivnesh/ISCLS-19/master/data/sample_snli.jsonl
!ls
file_name='sample_snli.jsonl'
json_data = []
with open(file_name) as data_file:
  for l in data_file:
    json_data.append(json.loads(l))
print('Dataset loaded.')


--2019-06-23 04:50:04--  https://raw.githubusercontent.com/Jivnesh/ISCLS-19/master/data/sample_snli.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50743 (50K) [text/plain]
Saving to: ‘sample_snli.jsonl.2’

sample_snli.jsonl.2 100%[===================>]  49.55K  --.-KB/s    in 0.02s   

2019-06-23 04:50:04 (1.97 MB/s) - ‘sample_snli.jsonl.2’ saved [50743/50743]

sample_data  sample_snli.jsonl	sample_snli.jsonl.1  sample_snli.jsonl.2
Dataset loaded.


In [70]:
json_data[0]

{'annotator_labels': ['neutral',
  'entailment',
  'neutral',
  'neutral',
  'neutral'],
 'captionID': '4705552913.jpg#2',
 'gold_label': 'neutral',
 'pairID': '4705552913.jpg#2r1n',
 'sentence1': 'Two women are embracing while holding to go packages.',
 'sentence1_binary_parse': '( ( Two women ) ( ( are ( embracing ( while ( holding ( to ( go packages ) ) ) ) ) ) . ) )',
 'sentence1_parse': '(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP are) (VP (VBG embracing) (SBAR (IN while) (S (NP (VBG holding)) (VP (TO to) (VP (VB go) (NP (NNS packages)))))))) (. .)))',
 'sentence2': 'The sisters are hugging goodbye while holding to go packages after just eating lunch.',
 'sentence2_binary_parse': '( ( The sisters ) ( ( are ( ( hugging goodbye ) ( while ( holding ( to ( ( go packages ) ( after ( just ( eating lunch ) ) ) ) ) ) ) ) ) . ) )',
 'sentence2_parse': '(ROOT (S (NP (DT The) (NNS sisters)) (VP (VBP are) (VP (VBG hugging) (NP (UH goodbye)) (PP (IN while) (S (VP (VBG holding) (S (VP (TO to) (

In [71]:
print(json_data[0]['gold_label'])
print(json_data[0]['sentence1'])
print(json_data[0]['sentence2'])

neutral
Two women are embracing while holding to go packages.
The sisters are hugging goodbye while holding to go packages after just eating lunch.


In [72]:
for i in range(5):
    print(json_data[i]['gold_label'])

neutral
entailment
contradiction
-
neutral


### Preprocessing started
 First datapoint is printed above for your  better understanding of how data looks. <br>
 Each data-sample has different attributes like annotator_labels, captionID, gold_label, pairID, sentence1, sentence2 etc. So we are interested in three attributes, namely, sentence1, sentence2 and gold_label.
The 'gold_label' field contains the label ('entailment', 'neutral' or 'contradiction') for each data point. There are some data points that don't have the gold_label (their value for the gold label field is '-'). We will only use those examples which have a gold label. Your job is to extract these three attributes (sentence1, sentence2 and gold_label) and preprocess it. <br>
 You need to complete obtain_data function. It has two parts.<br>


1.   Filter out data points without a gold label and extract each sentence pair into a triple and concatenate them altogether into a list. For example,   triple = (s1, s2, label).
2.  Build the set of tokens from collection of  all sentence pairs (sentence1 and sentence2 of each sample). For example, your set of tokens will look like { '2',
 '9',
 'A',
 'An',
 'Angeles',
 'Asian-themed',
 'At',
 'Chinese',
 'Doctors'}



     

In [73]:
def obtain_data(json_data):
    """
    Input:
        json_data: list (with each row of this list being a dictionary)
    
    Action:
    
        1. Ignore data points without a gold label
        2. Extract each sentence pair into a tuple of size 3 and concatenate them altogether into a list.
        
        i.e., for each item in json_data, store sentence1, sentence2 and gold_label values into 'example' list if gold_label != '-'
        
        example = (sentence1, sentence2, gold_label)
        append the 'example' tuple into list 'data' 
        data.append(example)
        
    Output:
        data: list of tuples of size 3
    """ 
    # YOUR CODE HERE
    data=[]
    for i in range(len(json_data)):
#       print(json_data[i]['gold_label'] in ['entailment', 'neutral' ,'contradiction'])
      if json_data[i]['gold_label'] in ['entailment', 'neutral' ,'contradiction']:
        ex=(json_data[i]['sentence1'],json_data[i]['sentence2'],json_data[i]['gold_label'])
        data.append(ex)
    return data
data = obtain_data(json_data)
print(data)

[('Two women are embracing while holding to go packages.', 'The sisters are hugging goodbye while holding to go packages after just eating lunch.', 'neutral'), ('Two women are embracing while holding to go packages.', 'Two woman are holding packages.', 'entailment'), ('Two women are embracing while holding to go packages.', 'The men are fighting outside a deli.', 'contradiction'), ('Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.', 'Two kids at a ballgame wash their hands.', 'neutral'), ('Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.', 'Two kids in jackets walk to school.', 'contradiction'), ('A man selling donuts to a customer during a world exhibition event held in the city of Angeles', 'A woman drinks her coffee in a small cafe.', 'contradiction'), ('A man 

In [74]:
'''test for obtain_data'''

'test for obtain_data'

In [75]:
def tokens(json_data):
    """
    Input:
        json_data: list (with each row of this list being a dictionary)
       
    Output: 
        voc_set: list, list of unique words in json_data (only consider those elements of json_data for which gold_label is not '-')
    """
    # YOUR CODE HERE
    d={}
    for i in range(len(json_data)):
      if json_data[i]['gold_label'] in ['entailment', 'neutral' ,'contradiction']:
        for word in json_data[i]['sentence1'].split():
          d[word]=1
        for word in json_data[i]['sentence2'].split():
          d[word]=1
    voc_set=set(d.keys())
    return voc_set
voc_set = tokens(json_data)
print(voc_set)

{'window', 'full', 'repainting', 'painted', 'serves', 'eating', 'baseball', 'plane.', 'rest', 'an', 'Kids', 'ride', 'wooden', 'her', 'white', 'camera', 'bus.', 'animal', 'symphony', 'Asian-themed', 'underwater.', 'jumping', 'next', 'ball', 'cheese', 'that', 'play', 'clouds,', 'ballgame', 'bowtie', 'favorite', 'tall', 'young', 'bikini', 'A', 'sand', 'movie.', 'a', 'mix', 'toy.', 'camera.', 'looking', 'amusement', 'Three', 'area,', 'football,', 'football', 'just', 'wait', 'patient.', 'propeller', 'scoring', 'The', 'cigarettes', 'crowd', 'race.', 'boy', 'for', 'outside', 'surrounding', 'catch', 'small', 'children', 'packages.', 'restaurant.', 'touch', 'chihuahua', 'having', 'cafe.', 'in', 'carrying', 'washing', 'women', 'curiosity.', 'side', 'head.', 'suit,', 'child', 'teams', 'waiting', 'lunch.', 'hugging', 'confronts', 'hair', 'bikes.', 'falls', 'men', 'standing', 'single', 'flowers', 'behind', 'An', 'his', 'garage', 'swimming', 'front', 'ball.', 'grilled', 'toy', 'food.', 'during', 'ma

In [76]:
'''test for tokens'''

'test for tokens'

## Build your vocabulary
Implement following three functions:

 
1.  **make_wtoi_dict**, word-to-integer is a python dictionary which can be used to map a word into a unique id. For example, stoi('hello') # This will give you an integer id  whose word map  is '*hello*'.
2. **itow**, integer-to-word is a python dictionary which can be used to map a given id into the corresponding word. For example, itos(100) # This will give you the word whose id is 100.
3.   Numericalize your data in the **process_data** function, namely convert the sentences into a list of ids and the labels into label_ids. (Use the stoi function and  label_id in the cell below for mapping.). Your output for ['Two women are embracing while holding to go packages.',
 'The sisters are hugging goodbye while holding to go packages after just eating lunch.',
 'neutral'] should look like ([44, 39, 13, 192, 231, 102, 141, 246, 168],
  [43, 133, 13, 151, 135, 231, 102, 141, 246, 71, 185, 170, 159, 178],
  1) 


In [77]:
def make_wtoi_dict(voc_set):
    """
    Input: 
        voc_set: list, list of unique words
        
    Output: 
         wtoi_dict: dictionary, keys are words in voc_set, values are unique integer ids for each word between 0 and len(voc_set)-1         
    """
    # YOUR CODE HERE
    wtoi_dict={}
    cnt=0
    for ele in voc_set:
      if ele not in wtoi_dict.keys():
        wtoi_dict[ele]=cnt
        cnt+=1
    return wtoi_dict
  
wtoi_dict = make_wtoi_dict(voc_set)
wtoi_dict['full']

1

In [78]:
def make_itow_dict(wtoi_dict):
    """
      Maps unique integer ids to words
      Input: 
          wtoi_dict: dictionary, contains word to integer mappings

      Output:
          itow_dict : dictionary, for each key, value in wtoi_dict - itow_dict should have a corresponding value, key pair

    """
    # YOUR CODE HERE
    itow_dict={}
    for key,value in wtoi_dict.items():
      itow_dict[value]=key
    return itow_dict
  
itow_dict = make_itow_dict(wtoi_dict)
itow_dict[1]

'full'

In [0]:
def wtoi(word, wtoi_dict):
  """
  Inputs:
    word: string, word for which id is required
    wtoi_dict: dictionary, contains word to int mappings
  Outputs:
    id : int, id corresponding to word from wtoi_dict if word is present in wtoi_dict.
         If word is not present in wtoi_dict, id = <unk-0>
  """
  # YOUR CODE HERE
  id=wtoi_dict.get(word,'<unk-0>')
  return id

In [80]:
print(wtoi("full", wtoi_dict))
print(wtoi("121213", wtoi_dict))

1
<unk-0>


In [0]:
def itow(id, itow_dict):
  """
  Inputs:
    id: int, integer id for which word is required
    itow_dict: dictionary, contains integer to word mappings
  Outputs:
    word: string, word corresponding to id from itow_dict, if id is present in itow_dict.
          If id is not present in itow_dict, word = <OOV>
  """
  # YOUR CODE HERE
  word=itow_dict.get(id,'<OOV>')
  return word

In [82]:
print(itow(1, itow_dict))
print(itow(3267637, itow_dict))

full
<OOV>


Numericalize your data in the **process_data** function, namely convert the sentences into a list of ids and the labels into label_ids. (Use the stoi function and  label_id in the cell below for mapping.). Your output for ['Two women are embracing while holding to go packages.',
 'The sisters are hugging goodbye while holding to go packages after just eating lunch.',
 'neutral'] may look like ([44, 39, 13, 192, 231, 102, 141, 246, 168],
  [43, 133, 13, 151, 135, 231, 102, 141, 246, 71, 185, 170, 159, 178],
  1) depending on your mappings

In [85]:
label_id = {'entailment': 0, 'neutral': 1, 'contradiction': 2}

def process_data(data, label_id):
    """
    Input:
        data: list, a list of 3-sized tuples
        label_id: a dictionary of length 3 (as defined above)
        
    Output:
        numerialized_data: list, list of 3-sized tuples
        
    Action:
        1. Convert each sentence pair into a tuple of size 3, (token_id_list, token_id_list, int) and concatenate them altogether into a list.   
        2. Out-of-vocabulary tokens should be mapped to '<unk-0>'.
        
        i.e., 
        
        data = [sentence1, sentence2, gold_label] is given
        a. Now for each word in sentence1, find unique integer id corresponding to that word using wtoi(word)
        b. append all the integers obtained into a list (say n_s1)
        c. do same thing for sentence2 and store it in n_s2
        d. Now using label_id, store integer value corresponding to gold_label in n_y 
        e. append n_s1, n_s2 and n_y in a list 'numerialized_data'
        f. return numerialized_data
    """
    # YOUR CODE HERE
    d=[]
    for tup in data:
        t=[]
        for word in tup[0].split():
          t.append(wtoi(word, wtoi_dict))
        curr=t
        t=[]
        for word in tup[1].split():
          t.append(wtoi(word, wtoi_dict))
        curr1=t
        d.append((curr,curr1,label_id[tup[2]]))
    return d
numerialized_data = process_data(data, label_id)
print(data[20])
numerialized_data[20]

('A young boy in a field of flowers carrying a ball', 'boy leaving baseball game', 'neutral')


([34, 32, 56, 69, 37, 197, 163, 89, 70, 37, 23], [56, 177, 6, 168], 1)